In [36]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
nltk.download('punkt')
nltk.download('stopwords')
df = pd.read_csv('/content/movie_review.csv')
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos


In [37]:
stop_words = set(stopwords.words('english'))
df["text"]=df["text"].apply(lambda text: " ".join([word for word in str(text).split() if word not in stop_words]))
df.head()


,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,"films adapted comic books plenty success , whe...",pos
1,0,cv000,29590,1,"starters , created alan moore ( eddie campbell...",pos
2,0,cv000,29590,2,say moore campbell thoroughly researched subje...,pos
3,0,cv000,29590,3,"book ( "" graphic novel , "" ) 500 pages long in...",pos
4,0,cv000,29590,4,"words , dismiss film source .",pos


In [38]:
df['text'] = df['text'].str.lower()
df.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,"films adapted comic books plenty success , whe...",pos
1,0,cv000,29590,1,"starters , created alan moore ( eddie campbell...",pos
2,0,cv000,29590,2,say moore campbell thoroughly researched subje...,pos
3,0,cv000,29590,3,"book ( "" graphic novel , "" ) 500 pages long in...",pos
4,0,cv000,29590,4,"words , dismiss film source .",pos


In [39]:
import string
df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted comic books plenty success whet...,pos
1,0,cv000,29590,1,starters created alan moore eddie campbell ...,pos
2,0,cv000,29590,2,say moore campbell thoroughly researched subje...,pos
3,0,cv000,29590,3,book graphic novel 500 pages long include...,pos
4,0,cv000,29590,4,words dismiss film source,pos


In [40]:
tokenized_text = [word_tokenize(text.lower()) for text in df['text']]
model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

In [41]:
vector_size = model.vector_size
review_vectors = []
for review in df['text']:
    words = word_tokenize(review.lower())
    words = [word for word in words if word in model.wv.key_to_index]
    if not words:
        review_vectors.append(np.zeros(vector_size))
    else:
        review_vectors.append(np.mean([model.wv[word] for word in words], axis=0))
df['review_vector'] = review_vectors
df['review_vector'].head()


0    [-0.48751113, 0.4033263, 0.31896687, -0.003969...
1    [-0.30769512, 0.12622319, 0.30849585, -0.12263...
2    [-0.4594829, 0.4418595, 0.3130667, -0.11118340...
3    [-0.31090054, 0.21570383, 0.2814854, -0.021491...
4    [-0.44113916, 0.18976526, 0.3095029, 0.3109031...
Name: review_vector, dtype: object

In [42]:
X = df['review_vector'].values
y = df['tag'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_stacked = np.vstack(X_train)
X_test_stacked = np.vstack(X_test)

logistic_model = LogisticRegression()
logistic_model.fit(X_train_stacked, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [43]:
y_pred = logistic_model.predict(X_test_stacked)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.566517305315204
Precision: 0.5667296230324241
Recall: 0.566517305315204
F1-score: 0.5615215184235944
